# Machine learning with Spark MLlib

  
Resources:
  - [Spark MLlib guide](https://spark.apache.org/docs/latest/ml-guide.html)
  - [Databricks MLlib guide](https://docs.databricks.com/spark/latest/mllib/index.html#)
  - [pyspark MLlib RDD API docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html)
  - [pyspark MLlib DataFrame API docs](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html)
  - [pyspark complete docs](https://spark.apache.org/docs/latest/api/python/)

## Part 1 : Machine learning with Spark MLlib using DataFrames

### loading data and explorion the dataset

In [0]:
# First we import necessary tools for our programm
import pyspark
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [0]:
# Loading the data set of credits and displaying it

credit_df = spark.read.load("/FileStore/tables/Credit_Card.csv", format="csv", sep=";", inferSchema= True, header=True)
display(credit_df)
print(type(credit_df))

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800,0
7,500000,1,1,2,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
8,100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542,0
9,140000,2,3,1,28,0,0,2,0,0,0,11285,14096,12108,12211,11793,3719,3329,0,432,1000,1000,1000,0
10,20000,1,3,2,35,-2,-2,-2,-2,-1,-1,0,0,0,0,13007,13912,0,0,0,13007,1122,0,0


<class 'pyspark.sql.dataframe.DataFrame'>

In [0]:
credit_df.printSchema()

root
-- ID: integer (nullable = true)
-- LIMIT_BAL: integer (nullable = true)
-- SEX: integer (nullable = true)
-- EDUCATION: integer (nullable = true)
-- MARRIAGE: integer (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_0: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: integer (nullable = true)
-- BILL_AMT2: integer (nullable = true)
-- BILL_AMT3: integer (nullable = true)
-- BILL_AMT4: integer (nullable = true)
-- BILL_AMT5: integer (nullable = true)
-- BILL_AMT6: integer (nullable = true)
-- PAY_AMT1: integer (nullable = true)
-- PAY_AMT2: integer (nullable = true)
-- PAY_AMT3: integer (nullable = true)
-- PAY_AMT4: integer (nullable = true)
-- PAY_AMT5: integer (nullable = true)
-- PAY_AMT6: integer (nullable = true)
-- default payment next month: integer (nullable = true)

In [0]:
# Explore data length and number of variables (columns)
print(" number of dataset lines : ", credit_df.count(), "number of dataset columns : ", len(credit_df.columns) )       
print(" number of variables : ", len(credit_df.columns)-1)     # 1 Column of ID   

number of dataset lines : 30000 number of dataset columns : 25
 number of variables : 24

In [0]:
# we change variable's name to a simplier one
credit_df = credit_df.withColumnRenamed("default payment next month", "DEFAULT_Y")
print("Column names:", credit_df.schema.names)

Column names: ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'DEFAULT_Y']

In [0]:
# we explore number of different valuescategories in each variable
credit_df.agg(*(countDistinct(col(c)).alias(c) for c in credit_df.columns[0:14])).show()   # the split is for clearer presentation
credit_df.agg(*(countDistinct(col(c)).alias(c) for c in credit_df.columns[14:25])).show()


+-----+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|
+-----+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+
30000| 81| 2| 7| 4| 56| 11| 11| 11| 11| 10| 10| 22723| 22346|
+-----+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+

+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT_Y|
+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
 22026| 21548| 21010| 20604| 7943| 7899| 7518| 6937| 6897| 6939| 2|
+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+

In [0]:
# Exploring Null values
from pyspark.sql.functions import isnan, when, count, col
credit_df.select([count(when(isnan(c), c)).alias(c) for c in credit_df.columns]).show()

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT_Y|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+

In [0]:
# checking on variables distinct values
credit_df.select("EDUCATION").distinct().sort("EDUCATION").show() ; credit_df.select("SEX",).distinct().show() ; 
credit_df.select("MARRIAGE",).distinct().show()

+---------+
EDUCATION|
+---------+
 0|
 1|
 2|
 3|
 4|
 5|
 6|
+---------+

+---+
SEX|
+---+
 1|
 2|
+---+

+--------+
MARRIAGE|
+--------+
 1|
 3|
 2|
 0|
+--------+

In [0]:
# cheking on variables distinct values
credit_df.select("PAY_0").distinct().sort("PAY_0").show()
credit_df.select("PAY_2").distinct().sort("PAY_2").show()
credit_df.select("PAY_5").distinct().sort("PAY_5").show()

+-----+
PAY_0|
+-----+
 -2|
 -1|
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
+-----+

+-----+
PAY_2|
+-----+
 -2|
 -1|
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
+-----+

+-----+
PAY_5|
+-----+
 -2|
 -1|
 0|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
+-----+

In [0]:
#affichage
#print(credit_df.select("ID", "LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_0", 
 #                 "BILL_AMT1", "BILL_AMT2", "DEFAULT_Y").show(5))

In [0]:
# Exploring number of differet modalities of target varibale "Default_Y"

print("number of peope with default (Default_Y == 1)):", credit_df.filter(credit_df.DEFAULT_Y == 1).count())
print("number of peope without default(Default_Y == 0)):", credit_df.filter(credit_df.DEFAULT_Y == 0).count())
print((credit_df.filter(credit_df.DEFAULT_Y == 1).count()/credit_df.filter(credit_df.DEFAULT_Y == 0).count())*100, " percentage of people with default in the total of clients")

number of peope with default (Default_Y == 1)): 6636
number of peope without default(Default_Y == 0)): 23364
28.40267077555213 percentage of people with default in the total of clients

In [0]:
# Exploring profile of clients with default by SEX
credit_df.select("sex", "Default_Y").groupBy("Default_Y", 'sex').count().show()

#credit_df.select("sex", "Default_Y")\
#.filter(((credit_df.filter(credit_df.DEFAULT_Y == 1).count() & credit_df.filter(credit_df.SEX == 1).count())/credit_df.filter(credit_df.DEFAULT_Y == 1).count())*100).show()

+---------+---+-----+
Default_Y|sex|count|
+---------+---+-----+
 1| 2| 3763|
 1| 1| 2873|
 0| 1| 9015|
 0| 2|14349|
+---------+---+-----+

### Training models and testing

In [0]:
# Drop ID column as we don't need it
credit_df = credit_df.drop("ID")

# Split data into training and test sample
splits = credit_df.randomSplit([0.7, 0.3], seed = 20)   
data_train = splits[0]
data_test = splits[1]


In [0]:
print(data_train.show(3))
print("Number of observations in data_train : " , data_train.count())
print("Number of observations in data_test : " , data_test.count())

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT_Y|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
 10000| 1| 1| 1| 38| -1| -1| 2| 0| 0| 2| 4711| 9617| 9307| 9665| 10470| 10169| 5000| 0| 500| 1100| 0| 1000| 1|
 10000| 1| 1| 1| 40| 2| -1| -1| -2| -2| -2| 3567| 1000| 0| 0| 0| 0| 5000| 0| 0| 0| 0| 0| 1|
 10000| 1| 1| 1| 42| 4| 3| 2| 2| 2| 0| 6744| 6494| 6247| 7975| 7701| 7787| 0| 0| 2000| 0| 359| 331| 0|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------+
only showing top 3 rows

None
Number of observations in data_train : 20985
Number of observations in data_test : 9015

In [0]:
# Get and convert categorical features (SEX, EDUCATION, MARRIAGE)
categorical_features = credit_df.schema.names[1:4]
catVect = VectorAssembler(inputCols = categorical_features, outputCol = "catFeatures")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")

# Get and normalize numerical features
numerical_features = credit_df.schema.names[0:1] + credit_df.schema.names[4:]
numVect = VectorAssembler(inputCols = numerical_features, outputCol = "numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol = "normFeatures")

In [0]:
# Creating pipeline
 
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol = "features")
pipeline = Pipeline(stages = [catVect, catIdx, numVect, minMax, featVect])
pipeline_object = pipeline.fit(data_train)

# Run training and test data through the pipeline
data_train = pipeline_object.transform(data_train).select("features", col("DEFAULT_Y").alias("target"))
data_test = pipeline_object.transform(data_test).select("features", col("DEFAULT_Y").alias("target"))

In [0]:
print(data_train.show(5))
print("Number of observations in data_train : " , data_train.count())
print("Number of observations in data_test : " , data_test.count())

+--------------------+------+
 features|target|
+--------------------+------+
[0.0,1.0,1.0,0.0,...| 1|
(24,[1,2,4,5,6,7,...| 1|
[0.0,1.0,1.0,0.0,...| 0|
[0.0,1.0,1.0,0.0,...| 0|
[0.0,1.0,1.0,0.0,...| 0|
+--------------------+------+
only showing top 5 rows

None
Number of observations in data_train : 20985
Number of observations in data_test : 9015

## Train and evaluate classification models
### Define evaluation metrics functions

In [0]:
accuracy = MulticlassClassificationEvaluator(labelCol = "target", predictionCol = "prediction", metricName = "accuracy")
precision = MulticlassClassificationEvaluator(labelCol = "target", predictionCol = "prediction", metricName = "weightedPrecision")
recall = MulticlassClassificationEvaluator(labelCol = "target", predictionCol = "prediction", metricName = "weightedRecall")

### Logistic regression

In [0]:
logit = LogisticRegression(labelCol = "target", featuresCol = "features", maxIter = 20, regParam = 0.2)
logit_model = logit.fit(data_train)
logit_predictions_df = logit_model.transform(data_test)

print("Accuracy: {:.4}".format(accuracy.evaluate(logit_predictions_df)))
print("Weighted precision: {:.4}".format(precision.evaluate(logit_predictions_df)))
print(" Weighted recall: {:.4}".format(recall.evaluate(logit_predictions_df)))

Accuracy: 0.98
Weighted precision: 0.9805
 Weighted recall: 0.98

### Random Forest

In [0]:
Rf = RandomForestClassifier(labelCol = "target", featuresCol = "features", maxDepth = 3, maxBins = 32, 
                            minInstancesPerNode = 1, minInfoGain=0.0, impurity = "gini", numTrees = 10, seed = 42) 
Rf_model = Rf.fit(data_train)
Rf_predictions_df = Rf_model.transform(data_test)

print("Accuracy: {:.4}".format(accuracy.evaluate(Rf_predictions_df)))
print("Weighted precision: {:.4}".format(precision.evaluate(Rf_predictions_df)))
print("Weighted recall: {:.4}".format(recall.evaluate(Rf_predictions_df)))

Accuracy: 0.9998
Weighted precision: 0.9998
Weighted recall: 0.9998

### Decision tree

In [0]:
tree = DecisionTreeClassifier(labelCol = "target", featuresCol = "features", maxDepth = 2, maxBins = 32, 
                              minInstancesPerNode = 1, minInfoGain = 0.0, impurity = "gini", seed = 42)
tree_model = tree.fit(data_train)
tree_predictions_df = tree_model.transform(data_test)

print("Accuracy: {:.4}".format(accuracy.evaluate(tree_predictions_df)))
print("Weighted precision: {:.4}".format(precision.evaluate(tree_predictions_df)))
print("Weighted recall: {:.4}".format(recall.evaluate(tree_predictions_df)))

Accuracy: 1.0
Weighted precision: 1.0
Weighted recall: 1.0

In [0]:
tp = int(Rf_predictions_df.filter("prediction == 1.0 AND target == 1").count())
fp = int(Rf_predictions_df.filter("prediction == 1.0 AND target == 0").count())
tn = int(Rf_predictions_df.filter("prediction == 0.0 AND target == 0").count())
fn = int(Rf_predictions_df.filter("prediction == 0.0 AND target == 1").count())

print("true positives:", tp)
print("false positives:", fp)
print("true negatives:", tn)
print("false negatives:", fn)

print("Accuracy: {:.4}".format((tp+tn)/(tp+fp+tn+fn)))
print("Precision: {:.4}".format((tp)/(tp+fp)))
print("Recall: {:.4}".format((tp)/(tp+fn)))

true positives: 1974
false positives: 0
true negatives: 7041
false negatives: 0
Accuracy: 1.0
Precision: 1.0
Recall: 1.0

## Part 2 :  Machine learning with Spark MLlib using RDDs

In [0]:
# re loading Data
credit_df2 = spark.read.load("/FileStore/tables/Credit_Card.csv", format="csv", sep=";", inferSchema= True, header=True)
#display(credit_df)
credit_df2.take(2)

Out[50]: [Row(ID=1, LIMIT_BAL=20000, SEX=2, EDUCATION=2, MARRIAGE=1, AGE=24, PAY_0=2, PAY_2=2, PAY_3=-1, PAY_4=-1, PAY_5=-2, PAY_6=-2, BILL_AMT1=3913, BILL_AMT2=3102, BILL_AMT3=689, BILL_AMT4=0, BILL_AMT5=0, BILL_AMT6=0, PAY_AMT1=0, PAY_AMT2=689, PAY_AMT3=0, PAY_AMT4=0, PAY_AMT5=0, PAY_AMT6=0, default payment next month=1),
 Row(ID=2, LIMIT_BAL=120000, SEX=2, EDUCATION=2, MARRIAGE=2, AGE=26, PAY_0=-1, PAY_2=2, PAY_3=0, PAY_4=0, PAY_5=0, PAY_6=2, BILL_AMT1=2682, BILL_AMT2=1725, BILL_AMT3=2682, BILL_AMT4=3272, BILL_AMT5=3455, BILL_AMT6=3261, PAY_AMT1=0, PAY_AMT2=1000, PAY_AMT3=1000, PAY_AMT4=1000, PAY_AMT5=0, PAY_AMT6=2000, default payment next month=1)]

In [0]:
# read as RDD
"""
credit_rdd = credit_df.rdd #session.read.csv("/FileStore/tables/Credit_Card.csv", header=True).rdd
print(type(credit_rdd))
print(credit_rdd)
credit_df.take(5)"""

<class 'pyspark.rdd.RDD'>
MapPartitionsRDD[1139] at javaToPython at NativeMethodAccessorImpl.java:0
Out[161]: [Row(LIMIT_BAL=20000, SEX=2, EDUCATION=2, MARRIAGE=1, AGE=24, PAY_0=2, PAY_2=2, PAY_3=-1, PAY_4=-1, PAY_5=-2, PAY_6=-2, BILL_AMT1=3913, BILL_AMT2=3102, BILL_AMT3=689, BILL_AMT4=0, BILL_AMT5=0, BILL_AMT6=0, PAY_AMT1=0, PAY_AMT2=689, PAY_AMT3=0, PAY_AMT4=0, PAY_AMT5=0, PAY_AMT6=0, DEFAULT_Y=1),
 Row(LIMIT_BAL=120000, SEX=2, EDUCATION=2, MARRIAGE=2, AGE=26, PAY_0=-1, PAY_2=2, PAY_3=0, PAY_4=0, PAY_5=0, PAY_6=2, BILL_AMT1=2682, BILL_AMT2=1725, BILL_AMT3=2682, BILL_AMT4=3272, BILL_AMT5=3455, BILL_AMT6=3261, PAY_AMT1=0, PAY_AMT2=1000, PAY_AMT3=1000, PAY_AMT4=1000, PAY_AMT5=0, PAY_AMT6=2000, DEFAULT_Y=1),
 Row(LIMIT_BAL=90000, SEX=2, EDUCATION=2, MARRIAGE=2, AGE=34, PAY_0=0, PAY_2=0, PAY_3=0, PAY_4=0, PAY_5=0, PAY_6=0, BILL_AMT1=29239, BILL_AMT2=14027, BILL_AMT3=13559, BILL_AMT4=14331, BILL_AMT5=14948, BILL_AMT6=15549, PAY_AMT1=1518, PAY_AMT2=1500, PAY_AMT3=1000, PAY_AMT4=1000, PAY_AMT5=1000, PAY_AMT6=5000, DEFAULT_Y=0),
 Row(LIMIT_BAL=50000, SEX=2, EDUCATION=2, MARRIAGE=1, AGE=37, PAY_0=0, PAY_2=0, PAY_3=0, PAY_4=0, PAY_5=0, PAY_6=0, BILL_AMT1=46990, BILL_AMT2=48233, BILL_AMT3=49291, BILL_AMT4=28314, BILL_AMT5=28959, BILL_AMT6=29547, PAY_AMT1=2000, PAY_AMT2=2019, PAY_AMT3=1200, PAY_AMT4=1100, PAY_AMT5=1069, PAY_AMT6=1000, DEFAULT_Y=0),
 Row(LIMIT_BAL=50000, SEX=1, EDUCATION=2, MARRIAGE=1, AGE=57, PAY_0=-1, PAY_2=0, PAY_3=-1, PAY_4=0, PAY_5=0, PAY_6=0, BILL_AMT1=8617, BILL_AMT2=5670, BILL_AMT3=35835, BILL_AMT4=20940, BILL_AMT5=19146, BILL_AMT6=19131, PAY_AMT1=2000, PAY_AMT2=36681, PAY_AMT3=10000, PAY_AMT4=9000, PAY_AMT5=689, PAY_AMT6=679, DEFAULT_Y=0)]

### create an RDD of vectors

In [0]:
#Transform our RDD of rows into an RDD of vectors
from pyspark.mllib.linalg import Vectors

credit_vectors = credit_df2.rdd.map(lambda row : Vectors.dense([item for item in row]))
print(type(credit_vectors))
credit_vectors.take(5)

<class 'pyspark.rdd.PipelinedRDD'>
Out[51]: [DenseVector([1.0, 20000.0, 2.0, 2.0, 1.0, 24.0, 2.0, 2.0, -1.0, -1.0, -2.0, -2.0, 3913.0, 3102.0, 689.0, 0.0, 0.0, 0.0, 0.0, 689.0, 0.0, 0.0, 0.0, 0.0, 1.0]),
 DenseVector([2.0, 120000.0, 2.0, 2.0, 2.0, 26.0, -1.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2682.0, 1725.0, 2682.0, 3272.0, 3455.0, 3261.0, 0.0, 1000.0, 1000.0, 1000.0, 0.0, 2000.0, 1.0]),
 DenseVector([3.0, 90000.0, 2.0, 2.0, 2.0, 34.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29239.0, 14027.0, 13559.0, 14331.0, 14948.0, 15549.0, 1518.0, 1500.0, 1000.0, 1000.0, 1000.0, 5000.0, 0.0]),
 DenseVector([4.0, 50000.0, 2.0, 2.0, 1.0, 37.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 46990.0, 48233.0, 49291.0, 28314.0, 28959.0, 29547.0, 2000.0, 2019.0, 1200.0, 1100.0, 1069.0, 1000.0, 0.0]),
 DenseVector([5.0, 50000.0, 1.0, 2.0, 1.0, 57.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 8617.0, 5670.0, 35835.0, 20940.0, 19146.0, 19131.0, 2000.0, 36681.0, 10000.0, 9000.0, 689.0, 679.0, 0.0])]

#### Draw some statistics on the RDD[vectors]

In [0]:
import numpy as np
from pyspark.mllib.stat import Statistics

In [0]:
# calculate summary statistics
summary = Statistics.colStats(credit_vectors)
print(summary.mean())  
print(summary.variance()) 

[ 1.50005000e+04 1.67484323e+05 1.60373333e+00 1.85313333e+00
 1.55186667e+00 3.54855000e+01 -1.67000000e-02 -1.33766667e-01
 -1.66200000e-01 -2.20666667e-01 -2.66200000e-01 -2.91100000e-01
 5.12233309e+04 4.91790752e+04 4.70131548e+04 4.32629490e+04
 4.03114010e+04 3.88717604e+04 5.66358050e+03 5.92116350e+03
 5.22568150e+03 4.82607687e+03 4.79938763e+03 5.21550257e+03
 2.21200000e-01]
[7.50025000e+07 1.68344557e+10 2.39247370e-01 6.24651004e-01
 2.72452264e-01 8.49697554e+01 1.26292987e+00 1.43325425e+00
 1.43249198e+00 1.36688512e+00 1.28411370e+00 1.32247154e+00
 5.42223996e+09 5.06570536e+09 4.80933754e+09 4.13871638e+09
 3.69629415e+09 3.54669172e+09 2.74342256e+08 5.30881709e+08
 3.10005092e+08 2.45428561e+08 2.33426624e+08 3.16038289e+08
 1.72276303e-01]
[30000. 30000. 30000. 29986. 29946. 30000. 15263. 14270. 14236. 13545.
 13053. 13714. 27992. 27494. 27130. 26805. 26494. 25980. 24751. 24604.
 24032. 23592. 23297. 22827. 6636.]

In [0]:
# Compute column summary statistics.
print(summary.numNonzeros()) 
print(summary.count())  # number of nonzeros in each column

[30000. 30000. 30000. 29986. 29946. 30000. 15263. 14270. 14236. 13545.
 13053. 13714. 27992. 27494. 27130. 26805. 26494. 25980. 24751. 24604.
 24032. 23592. 23297. 22827. 6636.]
30000

In [0]:
# We compute Pearson correlation 
print(Statistics.corr(credit_vectors, method="pearson"))

[[ 1.00000000e+00 2.61791577e-02 1.84974942e-02 3.91767348e-02
 -2.90794253e-02 1.86777345e-02 -3.05749323e-02 -1.12148683e-02
 -1.84935538e-02 -2.73488978e-03 -2.21992319e-02 -2.02701341e-02
 1.93886739e-02 1.79818975e-02 2.43540500e-02 4.03506029e-02
 1.67053032e-02 1.67296183e-02 9.74244346e-03 8.40612034e-03
 3.91505355e-02 7.79312973e-03 6.52187779e-04 2.99978072e-03
 -1.39519548e-02]
 [ 2.61791577e-02 1.00000000e+00 2.47552351e-02 -2.19160698e-01
 -1.08139410e-01 1.44712798e-01 -2.71214332e-01 -2.96382101e-01
 -2.86122954e-01 -2.67460010e-01 -2.49411395e-01 -2.35195399e-01
 2.85429865e-01 2.78314364e-01 2.83235784e-01 2.93987624e-01
 2.95562338e-01 2.90388951e-01 1.95235915e-01 1.78407954e-01
 2.10166748e-01 2.03242410e-01 2.17202432e-01 2.19595369e-01
 -1.53519876e-01]
 [ 1.84974942e-02 2.47552351e-02 1.00000000e+00 1.42319362e-02
 -3.13888401e-02 -9.08736465e-02 -5.76428789e-02 -7.07710032e-02
 -6.60960564e-02 -6.01732384e-02 -5.50638850e-02 -4.40077882e-02
 -3.36418696e-02 -3.11834722e-02 -2.45633117e-02 -2.18796791e-02
 -1.70051802e-02 -1.67331263e-02 -2.42454563e-04 -1.39090966e-03
 -8.59662491e-03 -2.22897151e-03 -1.66716180e-03 -2.76602228e-03
 -3.99605777e-02]
 [ 3.91767348e-02 -2.19160698e-01 1.42319362e-02 1.00000000e+00
 -1.43464340e-01 1.75060661e-01 1.05363998e-01 1.21565552e-01
 1.14024903e-01 1.08793456e-01 9.75201606e-02 8.23159864e-02
 2.35811807e-02 1.87487708e-02 1.30024007e-02 -4.51345309e-04
 -7.56650305e-03 -9.09895470e-03 -3.74561839e-02 -3.00381866e-02
 -3.99431409e-02 -3.82181661e-02 -4.03584549e-02 -3.71999031e-02
 2.80060777e-02]
 [-2.90794253e-02 -1.08139410e-01 -3.13888401e-02 -1.43464340e-01
 1.00000000e+00 -4.14169918e-01 1.99171905e-02 2.41990673e-02
 3.26876733e-02 3.31215489e-02 3.56291704e-02 3.43447660e-02
 -2.34718019e-02 -2.16017799e-02 -2.49094510e-02 -2.33438469e-02
 -2.53933980e-02 -2.12068248e-02 -5.97899978e-03 -8.09270284e-03
 -3.54135195e-03 -1.26592789e-02 -1.20475692e-03 -6.64094193e-03
 -2.43392157e-02]
 [ 1.86777345e-02 1.44712798e-01 -9.08736465e-02 1.75060661e-01
 -4.14169918e-01 1.00000000e+00 -3.94473762e-02 -5.01477762e-02
 -5.30484373e-02 -4.97216740e-02 -5.38259764e-02 -4.87734264e-02
 5.62385333e-02 5.42830744e-02 5.37097052e-02 5.13530932e-02
 4.93450481e-02 4.76126780e-02 2.61466791e-02 2.17848934e-02
 2.92473530e-02 2.13790056e-02 2.28499736e-02 1.94781530e-02
 1.38898343e-02]
 [-3.05749323e-02 -2.71214332e-01 -5.76428789e-02 1.05363998e-01
 1.99171905e-02 -3.94473762e-02 1.00000000e+00 6.72164383e-01
 5.74245093e-01 5.38840627e-01 5.09426064e-01 4.74553086e-01
 1.87068431e-01 1.89859109e-01 1.79785282e-01 1.79124771e-01
 1.80634625e-01 1.76980296e-01 -7.92687112e-02 -7.01005212e-02
 -7.05608377e-02 -6.40048889e-02 -5.81898859e-02 -5.86732144e-02
 3.24793728e-01]
 [-1.12148683e-02 -2.96382101e-01 -7.07710032e-02 1.21565552e-01
 2.41990673e-02 -5.01477762e-02 6.72164383e-01 1.00000000e+00
 7.66551683e-01 6.62067131e-01 6.22780245e-01 5.75500862e-01
 2.34886527e-01 2.35256943e-01 2.24145855e-01 2.22236512e-01
 2.21348373e-01 2.19403351e-01 -8.07007098e-02 -5.89899990e-02
 -5.59012313e-02 -4.68584115e-02 -3.70930798e-02 -3.65003755e-02
 2.63551202e-01]
 [-1.84935538e-02 -2.86122954e-01 -6.60960564e-02 1.14024903e-01
 3.26876733e-02 -5.30484373e-02 5.74245093e-01 7.66551683e-01
 1.00000000e+00 7.77358873e-01 6.86774511e-01 6.32683593e-01
 2.08472889e-01 2.37294534e-01 2.27494327e-01 2.27202281e-01
 2.25145193e-01 2.22326737e-01 1.29481433e-03 -6.67933957e-02
 -5.33107796e-02 -4.60665339e-02 -3.58630706e-02 -3.58610837e-02
 2.35252514e-01]
 [-2.73488978e-03 -2.67460010e-01 -6.01732384e-02 1.08793456e-01
 3.31215489e-02 -4.97216740e-02 5.38840627e-01 6.62067131e-01
 7.77358873e-01 1.00000000e+00 8.19835311e-01 7.16449482e-01
 2.02812069e-01 2.25816316e-01 2.44983138e-01 2.45917204e-01
 2.42902058e-01 2.39154130e-01 -9.36213575e-03 -1.94365661e-03
 -6.92352036e-02 -4.34614297e-02 -3.35895347e-02 -2.65650880e-02
 2.16613637e-01]
 [-2.21992319e-02 -2.49411395e-01 -5.5063

In [0]:
# Split data into Train and Test
(trainingData, testData) = credit_vectors.randomSplit([0.7, 0.3])
print(type(trainingData))

<class 'pyspark.rdd.PipelinedRDD'>

In [0]:
summary = Statistics.colStats(trainingData)
print(summary.count())  # number of nonzeros in each column
print(testData.count())
print(trainingData.count()+testData.count())

21052
8948
30000

#### remark : 
we saw that in order to run ML models, data should have LabelPoint type, otherwise models won't run.
So we change the type of our data from an RDD of vectors to LabelPoint

In [0]:
# LabeledPoint data trasnformation
from pyspark.mllib.regression import LabeledPoint
data_rdd = credit_df2.rdd.map(tuple)

Label_data1 = data_rdd.map(lambda line: LabeledPoint(line[-1],[line[1:-1]]))
Label_data1.take(5)

Out[64]: [LabeledPoint(1.0, [20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,-2.0,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(1.0, [120000.0,2.0,2.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,2.0,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0]),
 LabeledPoint(0.0, [90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0]),
 LabeledPoint(0.0, [50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0]),
 LabeledPoint(0.0, [50000.0,1.0,2.0,1.0,57.0,-1.0,0.0,-1.0,0.0,0.0,0.0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0])]

In [0]:
# Split data into Train and Test
(trainingL, testL) = Label_data1.randomSplit([0.7, 0.3])


In [0]:
#Little verification
print(trainingL.count(), testL.count(),trainingL.count()+testL.count())

20901 9099 30000

### DecisionTree

In [0]:
# Training DecisionTree model
from pyspark.mllib.tree import DecisionTree
tree_model= DecisionTree.trainClassifier(trainingL, 
                                     numClasses=2, 
                                     categoricalFeaturesInfo={})
tree_predictions =tree_model.predict(testL.map(lambda x: x.features))

In [0]:
# Evaluate our model
labelsAndPredictions_tree = testL.map(lambda lp: lp.label).zip(tree_predictions)
testMSE = labelsAndPredictions_tree.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /float(testL.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression Decision Tree :')
print(tree_model.toDebugString())

Test Mean Squared Error = 0.17925046708429498
Learned regression Decision Tree :
DecisionTreeModel classifier of depth 5 with 41 nodes
 If (feature 5 <= 1.5)
 If (feature 6 <= 1.5)
 Predict: 0.0
 Else (feature 6 > 1.5)
 If (feature 9 <= 1.0)
 Predict: 0.0
 Else (feature 9 > 1.0)
 If (feature 4 <= 51.5)
 If (feature 21 <= 0.5)
 Predict: 1.0
 Else (feature 21 > 0.5)
 Predict: 0.0
 Else (feature 4 > 51.5)
 If (feature 11 <= 1638.5)
 Predict: 0.0
 Else (feature 11 > 1638.5)
 Predict: 1.0
 Else (feature 5 > 1.5)
 If (feature 7 <= -0.5)
 If (feature 13 <= 214.0)
 If (feature 4 <= 30.5)
 If (feature 22 <= 71.0)
 Predict: 0.0
 Else (feature 22 > 71.0)
 Predict: 1.0
 Else (feature 4 > 30.5)
 If (feature 11 <= 493.5)
 Predict: 0.0
 Else (feature 11 > 493.5)
 Predict: 1.0
 Else (feature 13 > 214.0)
 If (feature 4 <= 48.5)
 Predict: 0.0
 Else (feature 4 > 48.5)
 If (feature 0 <= 25000.0)
 Predict: 0.0
 Else (feature 0 > 25000.0)
 Predict: 1.0
 Else (feature 7 > -0.5)
 If (feature 10 <= 1.0)
 If (feature 2 <= 4.5)
 If (feature 0 <= 18000.0)
 Predict: 0.0
 Else (feature 0 > 18000.0)
 Predict: 1.0
 Else (feature 2 > 4.5)
 If (feature 0 <= 55000.0)
 Predict: 1.0
 Else (feature 0 > 55000.0)
 Predict: 0.0
 Else (feature 10 > 1.0)
 If (feature 19 <= 10001.5)
 If (feature 16 <= 136.5)
 Predict: 0.0
 Else (feature 16 > 136.5)
 Predict: 1.0
 Else (feature 19 > 10001.5)
 If (feature 21 <= 2898.5)
 Predict: 0.0
 Else (feature 21 > 2898.5)
 Predict: 1.0

#### Random Forest

In [0]:
#Training the  RandomForest model
from pyspark.mllib.tree import RandomForest, RandomForestModel
Rf_model = RandomForest.trainRegressor(trainingL, categoricalFeaturesInfo={},
                                        numTrees=5)
Rf_predictions = Rf_model.predict(testL.map(lambda x: x.features))

In [0]:
# Evaluating our model

Rf_labelsAndPredictions = testL.map(lambda lp: lp.label).zip(Rf_predictions)
testMSE = Rf_labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /float(testL.count())
print('Test Mean Squared Error  RandomForestModel = ' + str(testMSE))
print('Learned regression gradient boosting:')
print(Rf_model.toDebugString())

Test Mean Squared Error RandomForestModel= 0.13513814759838047
Learned regression gradient boosting:
TreeEnsembleModel regressor with 5 trees

 Tree 0:
 If (feature 6 <= 1.5)
 If (feature 8 <= 0.5)
 If (feature 11 <= 1.5)
 If (feature 20 <= 0.5)
 Predict: 0.31735985533453887
 Else (feature 20 > 0.5)
 Predict: 0.1465798045602606
 Else (feature 11 > 1.5)
 If (feature 5 <= 1.5)
 Predict: 0.1224113184334632
 Else (feature 5 > 1.5)
 Predict: 0.6493955094991365
 Else (feature 8 > 0.5)
 If (feature 5 <= 1.5)
 If (feature 18 <= 1.5)
 Predict: 0.5061728395061729
 Else (feature 18 > 1.5)
 Predict: 0.276536312849162
 Else (feature 5 > 1.5)
 If (feature 12 <= 2253.5)
 Predict: 0.125
 Else (feature 12 > 2253.5)
 Predict: 0.7227722772277227
 Else (feature 6 > 1.5)
 If (feature 5 <= 1.5)
 If (feature 4 <= 40.5)
 If (feature 10 <= 1.0)
 Predict: 0.34152652005174644
 Else (feature 10 > 1.0)
 Predict: 0.48579545454545453
 Else (feature 4 > 40.5)
 If (feature 9 <= 1.0)
 Predict: 0.38380281690140844
 Else (feature 9 > 1.0)
 Predict: 0.6782608695652174
 Else (feature 5 > 1.5)
 If (feature 22 <= 6000.5)
 If (feature 9 <= 1.0)
 Predict: 0.6737357259380098
 Else (feature 9 > 1.0)
 Predict: 0.7735099337748345
 Else (feature 22 > 6000.5)
 If (feature 16 <= 28311.5)
 Predict: 0.21052631578947367
 Else (feature 16 > 28311.5)
 Predict: 0.632183908045977
 Tree 1:
 If (feature 5 <= 1.5)
 If (feature 9 <= 1.0)
 If (feature 6 <= 1.5)
 If (feature 22 <= 888.5)
 Predict: 0.18898541222750367
 Else (feature 22 > 888.5)
 Predict: 0.10449459510714963
 Else (feature 6 > 1.5)
 If (feature 22 <= 500.5)
 Predict: 0.4624413145539906
 Else (feature 22 > 500.5)
 Predict: 0.27
 Else (feature 9 > 1.0)
 If (feature 6 <= 1.5)
 If (feature 17 <= 3000.5)
 Predict: 0.2494061757719715
 Else (feature 17 > 3000.5)
 Predict: 0.3741258741258741
 Else (feature 6 > 1.5)
 If (feature 21 <= 2.5)
 Predict: 0.6136363636363636
 Else (feature 21 > 2.5)
 Predict: 0.4597315436241611
 Else (feature 5 > 1.5)
 If (feature 6 <= -0.5)
 If (feature 7 <= -0.5)
 If (feature 15 <= 1.0)
 Predict: 0.5454545454545454
 Else (feature 15 > 1.0)
 Predict: 0.1794871794871795
 Else (feature 7 > -0.5)
 If (feature 4 <= 37.5)
 Predict: 0.18181818181818182
 Else (feature 4 > 37.5)
 Predict: 1.0
 Else (feature 6 > -0.5)
 If (feature 9 <= 1.0)
 If (feature 21 <= 24485.0)
 Predict: 0.6769358867610324
 Else (feature 21 > 24485.0)
 Predict: 0.22727272727272727
 Else (feature 9 > 1.0)
 If (feature 6 <= 2.5)
 Predict: 0.793233082706767
 Else (feature 6 > 2.5)
 Predict: 0.6173913043478261
 Tree 2:
 If (feature 8 <= 1.5)
 If (feature 6 <= 1.5)
 If (feature 18 <= 1675.0)
 If (feature 20 <= 1999.5)
 Predict: 0.21360294117647058
 Else (feature 20 > 1999.5)
 Predict: 0.1284475965327029
 Else (feature 18 > 1675.0)
 If (feature 5 <= 1.5)
 Predict: 0.10258220698002825
 Else (feature 5 > 1.5)
 Predict: 0.633423180592992
 Else (feature 6 > 1.5)
 If (feature 14 <= 62265.5)
 If (feature 18 <= 5504.0)
 Predict: 0.4500846023688663
 Else (feature 18 > 5504.0)
 Predict: 0.15789473684210525
 Else (feature 14 > 62265.5)
 If (feature 5 <= 1.5)
 Predict: 0.36774193548387096
 Else (feature 5 > 1.5)
 Predict: 0.7716049382716049
 Else (feature 8 > 1.5)
 If (feature 5 <= 1.5)
 If (feature 6 <= 1.5)
 If (feature 9 <= 1.0)
 Predict: 0.25925925925925924
 Else (feature 9 > 1.0)
 Predict: 0.39361702127659576
 Else (feature 6 > 1.5)
 If (feature 21 <= 1998.0)
 Predict: 0.5384615384615384
 Else (feature 21 > 1998.0)
 Predict: 0.3736842105263158
 Else (feature 5 > 1.5)
 If (feature 19 <= 14544.5)
 If (feature 11 <= 266.5)
 Predict: 0.3076923076923077
 Else (feature 11 > 266.5)
 Predict: 0.7842157842157842
 Else (feature 19 > 14544.5)
 If (feature 0 <= 35000.0)
 Predict: 1.0
 Else (feature 0 > 35000.0)
 Predict: 0.0
 Tree 3:
 If (feature 5 <= 1.5)
 If (feature 6 <= 1.5)
 If (feature 19 <= 0.5)
 If (feature 17 <= 1785.5)
 Predict: 0.27206218564243256
 Else (feature 17 > 1785.5)
 Predict: 0.19242579324462641
 Else (feature 19 > 0.5)
 If (feature 0 <= 75000

### Gradient Boosting

In [0]:
# Training our Gradient Boost model

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
gbtModel = GradientBoostedTrees.trainClassifier(trainingL, 
                                     categoricalFeaturesInfo={})
gbt_predictions =gbtModel .predict(testL.map(lambda x: x.features))
labelsAndPredictions_gbt = testL.map(lambda lp: lp.label).zip(gbt_predictions)


In [0]:
# Evaluating our model

testMSE = labelsAndPredictions_gbt.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /float(testL.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression gradient boosting:')
print(gbtModel.toDebugString())

Test Mean Squared Error = 0.17782173865259918
Learned regression gradient boosting:
TreeEnsembleModel classifier with 100 trees

 Tree 0:
 If (feature 5 <= 1.5)
 If (feature 6 <= 1.5)
 If (feature 19 <= 0.5)
 Predict: -0.5287028227085315
 Else (feature 19 > 0.5)
 Predict: -0.7536128201459437
 Else (feature 6 > 1.5)
 If (feature 9 <= 1.0)
 Predict: -0.2702702702702703
 Else (feature 9 > 1.0)
 Predict: 0.0379746835443038
 Else (feature 5 > 1.5)
 If (feature 7 <= -0.5)
 If (feature 13 <= 214.0)
 Predict: 0.2653061224489796
 Else (feature 13 > 214.0)
 Predict: -0.38666666666666666
 Else (feature 7 > -0.5)
 If (feature 10 <= 1.0)
 Predict: 0.3333333333333333
 Else (feature 10 > 1.0)
 Predict: 0.5545243619489559
 Tree 1:
 If (feature 7 <= 0.5)
 If (feature 17 <= 2737.5)
 If (feature 11 <= 493.5)
 Predict: 0.10181575301004438
 Else (feature 11 > 493.5)
 Predict: -0.16669418887435763
 Else (feature 17 > 2737.5)
 If (feature 9 <= 1.0)
 Predict: -0.3771638775166224
 Else (feature 9 > 1.0)
 Predict: 0.19248265222372987
 Else (feature 7 > 0.5)
 If (feature 6 <= 1.5)
 If (feature 20 <= 701.5)
 Predict: 0.7151364622991233
 Else (feature 20 > 701.5)
 Predict: 0.12340484943889521
 Else (feature 6 > 1.5)
 If (feature 22 <= 0.5)
 Predict: 0.2611962808325061
 Else (feature 22 > 0.5)
 Predict: 0.018929008059729788
 Tree 2:
 If (feature 0 <= 75000.0)
 If (feature 9 <= 1.0)
 If (feature 18 <= 4267.0)
 Predict: 0.03185387203876906
 Else (feature 18 > 4267.0)
 Predict: -0.33933684405462516
 Else (feature 9 > 1.0)
 If (feature 6 <= 2.5)
 Predict: 0.35366030935204695
 Else (feature 6 > 2.5)
 Predict: -0.34399281801726056
 Else (feature 0 > 75000.0)
 If (feature 5 <= 0.5)
 If (feature 11 <= 3602.0)
 Predict: -0.09593209739370123
 Else (feature 11 > 3602.0)
 Predict: -0.3563171749959435
 Else (feature 5 > 0.5)
 If (feature 2 <= 3.5)
 Predict: 0.043605914682539104
 Else (feature 2 > 3.5)
 Predict: -1.4835823094393508
 Tree 3:
 If (feature 0 <= 75000.0)
 If (feature 18 <= 4997.5)
 If (feature 10 <= 1.0)
 Predict: 0.02349793517257165
 Else (feature 10 > 1.0)
 Predict: 0.26331338910557633
 Else (feature 18 > 4997.5)
 If (feature 16 <= 30293.5)
 Predict: -0.3921733731265574
 Else (feature 16 > 30293.5)
 Predict: 0.1874680295951006
 Else (feature 0 > 75000.0)
 If (feature 18 <= 738.5)
 If (feature 11 <= 0.5)
 Predict: 0.17012830531715847
 Else (feature 11 > 0.5)
 Predict: -0.09351208183286337
 Else (feature 18 > 738.5)
 If (feature 10 <= 1.0)
 Predict: -0.2932076840073219
 Else (feature 10 > 1.0)
 Predict: 0.08634515505621355
 Tree 4:
 If (feature 7 <= 0.5)
 If (feature 17 <= 4603.0)
 If (feature 9 <= 1.0)
 Predict: -0.11386289635706452
 Else (feature 9 > 1.0)
 Predict: 0.29572820766042685
 Else (feature 17 > 4603.0)
 If (feature 16 <= 191948.0)
 Predict: -0.33567834380872547
 Else (feature 16 > 191948.0)
 Predict: -0.009661412791388335
 Else (feature 7 > 0.5)
 If (feature 6 <= 1.5)
 If (feature 20 <= 701.5)
 Predict: 0.6155012728407563
 Else (feature 20 > 701.5)
 Predict: 0.1278830541564191
 Else (feature 6 > 1.5)
 If (feature 11 <= 249.5)
 Predict: -1.1279271058360194
 Else (feature 11 > 249.5)
 Predict: 0.07342253036247018
 Tree 5:
 If (feature 0 <= 145000.0)
 If (feature 8 <= 0.5)
 If (feature 10 <= 1.0)
 Predict: -0.0637714330068033
 Else (feature 10 > 1.0)
 Predict: 0.21630214883027385
 Else (feature 8 > 0.5)
 If (feature 5 <= 3.5)
 Predict: 0.1852402455538848
 Else (feature 5 > 3.5)
 Predict: -0.6770454634493228
 Else (feature 0 > 145000.0)
 If (feature 11 <= 6899.5)
 If (feature 20 <= 574.5)
 Predict: 0.07975531095289244
 Else (feature 20 > 574.5)
 Predict: -0.20152540423116963
 Else (feature 11 > 6899.5)
 If (feature 11 <= 143852.5)
 Predict: -0.37765682346660717
 Else (feature 11 > 143852.5)
 Predict: -0.12004726600337261
 Tree 6:
 If (feature 0 <= 75000.0)
 If (feature 18 <= 4997.5)
 If (feature 11 <= 16611.5)
 Predict: 0.16752411549686452
 Else (feature 11 > 16611.5)
 Predict: -0.011229513845697297
 Else (feature 18 > 4997.5)
 If (feature 14 <= 283

Whithing models presented above, we see that the model giving the best  RMSE score is RandomForest, with a RMSE = 0.1348481479422529.